In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path

pd.set_option('display.max_columns', 300)

In [ ]:
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Nottingham Forest', 'Tottenham', 'Sunderland', 'West Ham', 'Wolves']

In [ ]:
# fetch FPL data online
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])
fpl_online_df['team_name'] = [teams[i] for i in fpl_online_df['team']-1]
fpl_online_df['name'] = fpl_online_df.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_online_df['season'] = '25-26'
fpl_online_df

In [ ]:
fpl_online_df[fpl_online_df.team_name == 'Liverpool']

In [ ]:
[col for col in fpl_online_df.columns if 'defensive' in col]

In [ ]:
fpl_online_df[fpl_online_df.second_name.str.contains('Szobo')]

In [ ]:
# Statshead data
data1 = pd.read_csv('../../../misc/pl24-25_data1.csv', header=[0,1])

# edit column names that have unnamed main headers
new_columns = [('General',col[1]) if 'Unnamed' in col[0] \
                else col for col in data1.columns]
data1.columns = pd.MultiIndex.from_tuples(new_columns)

# remove duplicate columns, keeping the last occurrence
data1 = data1.loc[:,~data1.columns.duplicated(keep='last')] 

data1

In [ ]:
# Statshead data
data2 = pd.read_csv('../../../misc/pl24-25_data2.csv', header=[0,1])

# edit column names that have unnamed main headers
new_columns = [('General',col[1]) if 'Unnamed' in col[0] \
                else col for col in data2.columns]
data2.columns = pd.MultiIndex.from_tuples(new_columns)

# remove duplicate columns, keeping the last occurrence
data2 = data2.loc[:,~data2.columns.duplicated(keep='last')] 

data2

In [ ]:
data_combined = pd.concat([data1, data2], axis=0, ignore_index=True)
data_combined = data_combined[~data_combined.duplicated(subset=[('General', 'Player'), ('General', 'Date')], keep='first')]
data_combined

In [ ]:
data_combined['CBIT'] = data_combined[('General', 'Tkl+Int')] + data_combined[('Blocks', 'Blocks')] + data_combined[('General', 'Clr')]

In [ ]:
df = data_combined[(data_combined.CBIT>=10) & (data_combined[('General', 'Pos.')] == 'DF')]
df

In [ ]:
len(df[df[('General', 'Player')] == 'Murillo'])

In [ ]:
len(df[df[('General', 'Player')] == 'James Tarkowski'])

In [ ]:
len(df[df[('General', 'Player')] == 'Jan Bednarek'])

In [ ]:
len(df[df[('General', 'Player')] == 'Wout Faes'])

In [ ]:
len(data_combined[data_combined[('General', 'Player')] == 'Wout Faes'])

In [ ]:
len(data_combined[data_combined[('General', 'Player')] == 'Murillo'])

In [ ]:
len(data_combined[data_combined[('General', 'Player')] == 'Murillo'])

## fpl_df

In [ ]:
fpl_df = pd.read_csv('../data/fpl_df.csv', index_col=0)
fpl_df

In [ ]:
fpl_df[[col for col in fpl_online_df.columns if 'defensive' in col]]

In [ ]:
fpl_df[fpl_df.name.str.contains('Szobo').fillna(False)]

In [ ]:
col = 'defensive_contribution'
aux = fpl_df.groupby(['first_name', 'second_name', 'season'])[col].diff()
aux.notnull().sum()

In [ ]:
fpl_df.groupby(['first_name', 'second_name', 'season'])[col].mean().reset_index().tail(50)

In [ ]:
def my_fill_na(x, gameweek_col, diff_col):
    '''Fill nan values for first items for grouped variables where diff is calculated. But also don't fill for season 22-23,
    where data is missing for a number of weeks at the beginning of the season.'''
    my_value = x[diff_col] if (np.isnan(x[gameweek_col])) & (x['minutes']<=90) else x[gameweek_col]
    return my_value

In [ ]:
fpl_df[f'gameweek_{col}']

In [ ]:
fpl_df[[col for col in fpl_df.columns if 'defensive' in col]]